In [1]:
import cimgraph.data_profile.cimhub_2023 as cim
from uuid import UUID
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import BlazegraphConnection
from cimgraph.models import FeederModel
from cimgraph.models import DistributedArea
import cimgraph.queries.sparql as sparql
import cimgraph.utils as utils
from defusedxml.ElementTree import parse
cim_profile = 'cimhub_2023'

In [3]:
from cimgraph.databases.fileparsers import XMLFile

params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee13.xml", cim_profile=cim_profile, iec61970_301=8)
# params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee123_pv.xml", cim_profile=cim_profile, iec61970_301=8)
# params = ConnectionParameters(filename="/home/ande188/CIM-Graph/tests/test_models/ieee9500bal.xml", cim_profile=cim_profile, iec61970_301=8)

xml_file = XMLFile(params)

graph2 = xml_file.create_new_graph({})


In [ ]:
# %load_ext line_profiler

# # def create_new_graph():
# #     xml_file.create_new_graph({})

# %lprun -f xml_file.create_new_graph xml_file.create_new_graph({})

In [ ]:
import cProfile

cProfile.run('xml_file.create_new_graph({})',sort=1)

In [4]:
feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
# feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507" #123 node
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=xml_file, container=feeder, distributed=False)

In [5]:
network.pprint(cim.ACLineSegment)

{
    "a04cdfb1-e951-4fc4-8882-0323cd70ae3c": {
        "identifier": "a04cdfb1-e951-4fc4-8882-0323cd70ae3c",
        "mRID": "A04CDFB1-E951-4FC4-8882-0323CD70AE3C",
        "name": "650632",
        "Terminals": "['000BA1F2-6879-47D7-826D-3CACD23EE20B', '3589A9BE-F3BE-45C3-B84E-3CE5B6105D11']",
        "length": "609.6",
        "ACLineSegmentPhases": "['2A5A1025-AD81-4BFA-A7F0-ED70F7F43DAA', '2D60B241-DE87-48BD-A8D3-FE4B705B40ED', 'A66508A9-C9C1-4DB5-8FFD-F081E5E9E14D']"
    },
    "abf53597-a808-422a-b7ee-552f24d83a5f": {
        "identifier": "abf53597-a808-422a-b7ee-552f24d83a5f",
        "mRID": "ABF53597-A808-422A-B7EE-552F24D83A5F",
        "name": "632670",
        "Terminals": "['5EDB2918-D669-457C-9F44-E08EBA5AC850', 'B554744A-F5B5-485C-B67E-2487A8740295']",
        "length": "203.3016",
        "ACLineSegmentPhases": "['4894C0CB-632C-454A-B9A2-1C2E41FB25FF', 'CF8E1FD4-BF1D-49B6-905C-8D0F9587FEF7', '28DAC6BE-8DD5-4C61-958E-ADB4E8A91EF8']"
    },
    "45395c84-f20a-4f5a-977f-

In [6]:
network.pprint(cim.Terminal)

{
    "a55ac772-d3f2-48af-8c84-e80f43198c72": {
        "identifier": "a55ac772-d3f2-48af-8c84-e80f43198c72",
        "mRID": "A55AC772-D3F2-48AF-8C84-E80F43198C72",
        "name": "school_T1",
        "sequenceNumber": "1"
    },
    "337f7a56-c530-4f05-8586-940e8460993a": {
        "identifier": "337f7a56-c530-4f05-8586-940e8460993a",
        "mRID": "337F7A56-C530-4F05-8586-940E8460993A",
        "name": "house_T1",
        "sequenceNumber": "1"
    },
    "90d2243e-05d4-432c-a23b-a6b8d732b027": {
        "identifier": "90d2243e-05d4-432c-a23b-a6b8d732b027",
        "mRID": "90D2243E-05D4-432C-A23B-A6B8D732B027",
        "name": "school_T1",
        "sequenceNumber": "1"
    },
    "a13ee73d-da09-44d8-9008-7d842f18cb48": {
        "identifier": "a13ee73d-da09-44d8-9008-7d842f18cb48",
        "mRID": "A13EE73D-DA09-44D8-9008-7D842F18CB48",
        "name": "batidle_T1",
        "sequenceNumber": "1"
    },
    "a4b4c25c-8744-4b8b-a612-19b80afa110d": {
        "identifier": "a4b4c25c-

In [ ]:
graph = {}
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=7)
database = BlazegraphConnection(params)

In [ ]:
rdf = '''{http://www.w3.org/1999/02/22-rdf-syntax-ns#}'''
# Parse the XML file
# tree = parse('./test_models/ieee13_assets.xml')
tree = parse('/home/ande188/CIM-Graph/tests/test_models/ieee9500bal.xml')
root = tree.getroot()

class_index = {}

# # Function to extract tag without namespace
# def extract_tag(tag):
#     return tag.split('}')[-1]

# Iterate over the elements and create dataclass instances
for element in root:

    class_name = element.tag.split('{'+params.namespace+'}')[1]
    cim_class = eval(f'cim.{class_name}')

    if class_name in cim.__all__:
        uri = element.get(f'{rdf}about')
        identifier = UUID(uri.strip('_').lower())
        uri = uri.split(':')[-1]  # Extract UUID from the full URI
        class_index[identifier] = cim_class
        obj = database.create_object(graph, cim_class, uri)

# for element in root:      
#     class_name = element.tag.split('{'+params.namespace+'}')[1]
#     cim_class = eval(f'cim.{class_name}')

#     if class_name in cim.__all__:
#         uri = element.get(f'{rdf}about')
        
#         identifier = UUID(uri.strip('_').lower())
#         uri = uri.split(':')[-1]  # Extract UUID from the full URI
        
#         obj = graph[cim_class][UUID(uri)]
#         for sub_element in element:
#             sub_tag = sub_element.tag.split('}')[-1]
#             association = database.check_attribute(cim_class, sub_tag)
#             try:
#                 edge_uri = sub_element.attrib[f'{rdf}resource'].split('uuid:')[-1]
#             except:
#                 edge_uri = None

#             if edge_uri is not None:
#                 if database.namespace not in edge_uri:
#                     edge_uuid = UUID(edge_uri.strip('_').lower())                  
#                     edge_class = class_index[edge_uuid]
#                     database.create_edge(graph, cim_class, identifier, sub_tag, edge_class, edge_uri)
#                     reverse = cim_class.__dataclass_fields__[association].metadata['inverse']
#                     database.create_edge(graph, edge_class, edge_uuid, reverse, cim_class, graph[cim_class][identifier].uri())
#                 else:
#                     enum_text = edge_uri.split(database.namespace)[1]
#                     enum_text = enum_text.split('>')[0]
#                     enum_class = enum_text.split('.')[0]
#                     enum_value = enum_text.split('.')[1]
#                     edge_enum = eval(f'cim.{enum_class}(enum_value)')
#                     if association is not None:
#                         setattr(graph[cim_class][identifier], association, edge_enum)
#             else:
#                 if association is not None:
#                     setattr(obj, association, sub_element.text)


In [ ]:
len(graph[cim.ACLineSegment].keys())

In [ ]:
import multiprocessing
import defusedxml.ElementTree as ET

# Parse the XML file
def parse_xml_file(file_path):
    tree = ET.parse(file_path)
    return tree.getroot()

# Extract tag without namespace
def extract_tag(tag):
    return tag.split('}')[-1]

# Function to create the object and set attributes
def process_element(element, element_to_class, graph, lock):
    tag = extract_tag(element.tag)
    data_class = element_to_class.get(tag)
    if data_class:
        uuid = element.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}about')
        uuid = uuid.split(':')[-1]  # Extract UUID from the full URI
        with lock:
            if uuid not in graph:
                graph[uuid] = xml_file.create_object(graph, data_class, uuid) #data_class(mRID=uuid)
        obj = graph[uuid]

        data_class in cim.__all__
        
        for sub_element in element:
            sub_tag = extract_tag(sub_element.tag)
            if sub_tag.startswith('IdentifiedObject.'):
                attribute = sub_tag.split('.')[-1]
                with lock:
                    setattr(obj, attribute, sub_element.text)
                    
            elif sub_tag.startswith(tag + '.'):
                attribute = sub_tag.split('.')[-1]
                with lock:
                    setattr(obj, attribute, sub_element.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource'))


In [ ]:
def parse_nodes(graph, element, lock):
        # Iterate over the elements and create dataclass instances
        # for element in self.root:
        class_name = element.tag.split('{'+self.namespace+'}')[1]

        if class_name in self.cim.__all__:
            print(class_name)
            cim_class = eval(f'self.cim.{class_name}')
            uri = element.get(f'{self.rdf}about')
            identifier = UUID(uri.strip('_').lower())
            uri = uri.split(':')[-1]  # Extract UUID from the full URI
            self.class_index[identifier] = cim_class
            with lock:
                obj = self.create_object(graph, cim_class, uri)
        else:
            # _log.warning(f'{class_name} not in data profile')
            pass
        return None

In [ ]:
def main():
    # Define the mapping from XML element names to dataclass types
    element_to_class = {
        'GeographicalRegion': cim.GeographicalRegion,
        'SubGeographicalRegion': cim.SubGeographicalRegion,
        'Substation': cim.Substation,
        'Location': cim.Location,
        'Feeder': cim.Feeder
    }

    xml_file_path = '/home/ande188/CIM-Graph/tests/test_models/ieee13.xml'
    root = parse_xml_file(xml_file_path)
    
    with multiprocessing.Manager() as manager:
        graph = manager.dict()
        lock = manager.Lock()

        # # Use ProcessPoolExecutor to process elements in parallel
        # with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        #     pool.starmap(
        #         process_element, 
        #         [(element, element_to_class, graph, lock) for element in root]
        #     )
        with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
            pool.starmap(parse_nodes, [(graph, element, lock) for element in root])
            
        # Example of accessing created objects (for debugging or verification)
        for uuid, obj in graph.items():
            print(uuid, obj)

In [ ]:
main()